In [5]:
from openai import OpenAI
import base64
import os
import requests
import pandas as pd
import io
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

# An Example Page

A demonstration of parsing of one page. 

In [53]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../../data/primary_sources/Paris_1878_Italy_2.jpg"

In [54]:
# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [55]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Parse this image to .csv with \t as a delimiter and the variables Number, Name, Place, Description."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 2500
}

In [56]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9cAsb9BaWEzzqJK6DFPYrQ5sF0m8n', 'object': 'chat.completion', 'created': 1718885313, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Here is the parsed content in CSV format. The delimiter used is a tab (`\\t`).\n\n```csv\nNumber\tName\tPlace\tDescription\n\tPodesta (E.)\tprofesseur, à Arezzo\tTraité élementaire d\'agriculture; premiers éléments d\'agriculture pour les écoles élémentaires.\n37\tRosalba (C.)\tarchitecte, à Avellino\tProjet d\'un édifice scolaire municipal constrait à Santa-Maria, Capua-Vetere.\n42\tSanti (A.)\tà Murano (Venise)\tRelations sur l\'instruction et l\'éducation dans la commune de Murano.\n38\tScaramuglia (T.)\tprofesseur, à Gualdo-Tadino (Pérouse)\tDessins à l\'aquarelle par les élèves de l\'école technique de Gualdo-Tadino (album).\n33\tSociété centrale ouvrière napolitaine\tà Naples\tDessins d\'ornements, de figures, de géometrie, de machines (1 toile et 4 albums).\n32\tSoli (P.)\tarchitec

In [66]:
csv_content = response.json()['choices'][0]['message']['content']

# Remove markdown and extract CSV string
csv_string = csv_content.split('```\n')[1].split('```')[0]
# Or:
csv_string = csv_content.split('```csv\n')[1].split('```')[0]

"Number\tName\tPlace\tDescription\n\tPodesta (E.)\tprofesseur, à Arezzo\tTraité élementaire d'agriculture; premiers éléments d'agriculture pour les écoles élémentaires.\n37\tRosalba (C.)\tarchitecte, à Avellino\tProjet d'un édifice scolaire municipal constrait à Santa-Maria, Capua-Vetere.\n42\tSanti (A.)\tà Murano (Venise)\tRelations sur l'instruction et l'éducation dans la commune de Murano.\n38\tScaramuglia (T.)\tprofesseur, à Gualdo-Tadino (Pérouse)\tDessins à l'aquarelle par les élèves de l'école technique de Gualdo-Tadino (album).\n33\tSociété centrale ouvrière napolitaine\tà Naples\tDessins d'ornements, de figures, de géometrie, de machines (1 toile et 4 albums).\n32\tSoli (P.)\tarchitecte, à Milan\tProjets d'écoles élémentaires.\n26\tSotis (Dr J.)\tà Fondi (Caserte)\tConferences sur l'hygiène de l'habitant de la campagne et sur l'agriculture pratique.\n44\tThevenet (J.)\tà Milan\tCours complet d'écriture (5 modèles).\n45\tZanetti (Abbé V.)\tdirecteur du Musée verrier et de l'éco

In [67]:
df = pd.read_csv(io.StringIO(csv_string), delimiter='\t')

In [68]:
df['Pdf'] = os.path.basename(image_path)

# A function

A function to loop through all files and export jsons of raw outputs and ready-made csv's per page. 

In [71]:
def chatgpt_csv(file_path):
    # Encode the image
    # Getting the base64 string
    base64_image = encode_image(file_path)

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }
    # Construct query to ChatGPT
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Parse this image to .csv with \t as a delimiter and the variables Number, Name, Place, Description."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 2500
    }
    # Obtain response
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    # Obtain .csv content and parse
    csv_content = response.json()['choices'][0]['message']['content']
    # Remove markdown and extract CSV string
    try:
        csv_string = csv_content.split('```csv\n')[1].split('```')[0]
    except:
        csv_string = csv_content.split('```\n')[1].split('```')[0]
    # Parse the csv in a df
    df = pd.read_csv(io.StringIO(csv_string), delimiter='\t')
    # Add the filename
    df['Pdf'] = os.path.basename(file_path)
    # Export to directory
    df.to_csv("../../data/gpt_ocr/" + os.path.basename(file_path) + ".csv", 
              sep="\t",
              index=False)  
    
    
    

In [72]:
chatgpt_csv("../../data/primary_sources/Paris_1878_Italy_3.jpg")
# Debug from Italy 3, Italy 3 doens't parse correctly yet